Universidad Torcuato Di Tella

Metodos computacionales\
**Trabajo Práctico 1:
Resoluciones Numéricas de Ecuaciones Diferenciales**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def f_piecewise(x):
    out = np.zeros_like(x)
    m = (0 <= x) & (x < 0.25);  out[m] = 4.0*x[m]
    m = (0.25 <= x) & (x < 0.50); out[m] = -2.0*x[m] + 1.5
    m = (0.50 <= x) & (x < 0.75); out[m] =  2.0*x[m] - 0.5
    m = (0.75 <= x) & (x <= 1.00); out[m] = -4.0*x[m] + 4.0
    return out

def heat_explicit(alpha, f, dx, dt, T):
    N  = int(round(1.0/dx))
    x  = np.linspace(0.0, 1.0, N+1)
    Nt = int(round(T/dt))
    r  = alpha * dt / (dx*dx)
    U  = np.zeros((Nt+1, N+1))
    U[0, :] = f(x)
    for n in range(Nt):
        Un = U[n, :]
        U[n+1, 1:-1] = Un[1:-1] + r*(Un[2:] - 2.0*Un[1:-1] + Un[:-2])
    return U, x, np.linspace(0.0, T, Nt+1), r

def _thomas(a, b, c, d):
    n = len(b)
    ac, bc, cc, dc = a.copy(), b.copy(), c.copy(), d.copy()
    for i in range(1, n):
        m = ac[i-1] / bc[i-1]
        bc[i] -= m * cc[i-1]
        dc[i] -= m * dc[i-1]
    x = np.zeros(n)
    x[-1] = dc[-1] / bc[-1]
    for i in range(n-2, -1, -1):
        x[i] = (dc[i] - cc[i] * x[i+1]) / bc[i]
    return x

def heat_implicit(alpha, f, dx, dt, T):
    N  = int(round(1.0/dx))
    x  = np.linspace(0.0, 1.0, N+1)
    Nt = int(round(T/dt))
    r  = alpha * dt / (dx*dx)
    U  = np.zeros((Nt+1, N+1))
    U[0, :] = f(x)
    m = N - 1
    a = -r * np.ones(m-1)
    b = (1 + 2*r) * np.ones(m)
    c = -r * np.ones(m-1)
    for n in range(Nt):
        rhs = U[n, 1:-1].copy()
        U[n+1, 1:-1] = _thomas(a, b, c, rhs)
    return U, x, np.linspace(0.0, T, Nt+1), r

# Ejecución con los parámetros del TP
if __name__ == "__main__":
    alpha, dx, dt, T = 1.0, 0.05, 0.001, 0.1
    Ue, x, t, r = heat_explicit(alpha, f_piecewise, dx, dt, T)
    Ui, x, t, _ = heat_implicit(alpha, f_piecewise, dx, dt, T)
    # Guardado a CSV y GIF: ver notebook o script adjunto.



: 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
TP1 - Parte 1: Ecuación del calor 1D (Dirichlet)
- Derivación de esquemas (explícito/implícito)
- Implementación en Python
- Ejecución con parámetros del enunciado
- Export a CSV y GIF de la evolución temporal
"""
from matplotlib.animation import FuncAnimation, PillowWriter
import csv
from typing import Tuple

def f_piecewise(x: np.ndarray) -> np.ndarray:
    """Condición inicial por tramos en [0,1]."""
    out = np.zeros_like(x)
    m = (0 <= x) & (x < 0.25);   out[m] = 4.0 * x[m]
    m = (0.25 <= x) & (x < 0.50); out[m] = -2.0 * x[m] + 1.5
    m = (0.50 <= x) & (x < 0.75); out[m] =  2.0 * x[m] - 0.5
    m = (0.75 <= x) & (x <= 1.00);out[m] = -4.0 * x[m] + 4.0
    return out
np.zeros((3, 5), dtype = float)

def heat_explicit(alpha: float, f, dx: float, dt: float, T: float):
    """Forward Euler en tiempo, centrado en espacio. Dirichlet 0 en bordes."""
    N  = int(round(1.0/dx))
    x  = np.linspace(0.0, 1.0, N+1)
    Nt = int(round(T/dt))
    t  = np.linspace(0.0, T, Nt+1)
    r  = alpha * dt / (dx*dx)
    U  = np.zeros((Nt+1, N+1))
    U[0, :] = f(x)
    for n in range(Nt):
        Un = U[n, :]
        U[n+1, 1:-1] = Un[1:-1] + r * (Un[2:] - 2.0*Un[1:-1] + Un[:-2])
        # U[n+1, 0] = U[n+1, -1] = 0 (Dirichlet homogéneas por construcción)
    return U, x, t, r

def _thomas(a: np.ndarray, b: np.ndarray, c: np.ndarray, d: np.ndarray) -> np.ndarray:
    """Resuelve Ax=d para A tridiagonal con subdiag=a, diag=b, superdiag=c."""
    n = len(b)
    ac, bc, cc, dc = a.copy(), b.copy(), c.copy(), d.copy()
    for i in range(1, n):
        m = ac[i-1] / bc[i-1]
        bc[i] -= m * cc[i-1]
        dc[i] -= m * dc[i-1]
    x = np.zeros(n)
    x[-1] = dc[-1] / bc[-1]
    for i in range(n-2, -1, -1):
        x[i] = (dc[i] - cc[i] * x[i+1]) / bc[i]
    return x

def heat_implicit(alpha: float, f, dx: float, dt: float, T: float):
    """Backward Euler en tiempo, centrado en espacio. Dirichlet 0 en bordes."""
    N  = int(round(1.0/dx))
    x  = np.linspace(0.0, 1.0, N+1)
    Nt = int(round(T/dt))
    t  = np.linspace(0.0, T, Nt+1)
    r  = alpha * dt / (dx*dx)
    U  = np.zeros((Nt+1, N+1))
    U[0, :] = f(x)
    m = N - 1
    a = -r * np.ones(m-1)         # subdiagonal
    b = (1 + 2*r) * np.ones(m)    # diagonal
    c = -r * np.ones(m-1)         # superdiagonal
    for n in range(Nt):
        rhs = U[n, 1:-1].copy()   # sin términos de borde (Dirichlet homogéneas)
        U[n+1, 1:-1] = _thomas(a, b, c, rhs)
    return U, x, t, r

def save_matrix_csv(path: str, U: np.ndarray, x: np.ndarray, t: np.ndarray) -> None:
    """Guarda U (tiempo×espacio) con encabezado 't,x=...'. Sin dependencias externas."""
    header = ["t"] + [f"x={xi:.2f}" for xi in x]
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(header)
        for i in range(len(t)):
            row = [f"{t[i]:.6f}"] + [f"{val:.8f}" for val in U[i, :]]
            w.writerow(row)

def make_gif(path: str, x: np.ndarray, U_exp: np.ndarray, U_imp: np.ndarray, dt: float) -> None:
    """Genera un GIF comparando perfiles explícito vs implícito a través del tiempo."""
    fig, ax = plt.subplots(figsize=(6, 4))
    (line_exp,) = ax.plot(x, U_exp[0, :], label="Explícito")
    (line_imp,) = ax.plot(x, U_imp[0, :], label="Implícito")
    ax.set_xlabel("x"); ax.set_ylabel("u(x,t)")
    ax.legend(loc="best")
    ax.set_xlim(0, 1)
    ymin = min(U_exp.min(), U_imp.min())
    ymax = max(U_exp.max(), U_imp.max())
    pad = 0.05 * (ymax - ymin if ymax > ymin else 1.0)
    ax.set_ylim(ymin - pad, ymax + pad)

    Nt = U_exp.shape[0] - 1
    def update(frame):
        line_exp.set_ydata(U_exp[frame, :])
        line_imp.set_ydata(U_imp[frame, :])
        ax.set_title(f"Ecuación del calor 1D (frame {frame}/{Nt})")
        return (line_exp, line_imp)

    ani = FuncAnimation(fig, update, frames=Nt+1, interval=30, blit=True)
    ani.save(path, writer=PillowWriter(fps=5))
    plt.close(fig)

def main():
    # Parámetros del enunciado
    alpha = 1.0
    dx    = 0.05
    dt    = 0.001
    T     = 0.1

    # Cálculo
    Ue, x, t, r_exp = heat_explicit(alpha, f_piecewise, dx, dt, T)
    Ui, x, t, r_imp = heat_implicit(alpha, f_piecewise, dx, dt, T)

    # Export CSV (tiempo×espacio)
    save_matrix_csv("U_explicit.csv", Ue, x, t)
    save_matrix_csv("U_implicit.csv", Ui, x, t)

    # GIF
    make_gif("heat_1D_exp_vs_imp.gif", x, Ue, Ui, dt)

    # Diagnóstico
    diff_L2 = np.linalg.norm(Ue[-1, :] - Ui[-1, :]) * np.sqrt(dx)
    print(f"r (explícito) = {r_exp:.3f} (estable si r<=0.5 en 1D)")
    print(f"r (implícito) = {r_imp:.3f} (A-estable)")
    print("Archivos generados: U_explicit.csv, U_implicit.csv, heat_1D_exp_vs_imp.gif")
    print(f"Norma L2 de la diferencia en t=T: {diff_L2:.6e}")

if __name__ == "__main__":
    main()


r (explícito) = 0.400 (estable si r<=0.5 en 1D)
r (implícito) = 0.400 (A-estable)
Archivos generados: U_explicit.csv, U_implicit.csv, heat_1D_exp_vs_imp.gif
Norma L2 de la diferencia en t=T: 2.333431e-03


In [ ]:
def condicion_inicial(x):
    if 0 <= x and (x < 0.25):
        return 4.0*x
    elif (0.25 <= x) and (x < 0.50):
        return -2.0*x + 1.5
    elif 0.50 <= x and x < 0.75:
        return 2.0*x - 0.5
    elif 0.75 <= x and x <= 1.00:
        return -4.0*x + 4.0
    else:
        print("x tiene que tenr un valor entre 0 y 1")
                       


def e_c_metodoexplicito(alpha, funcion, dx, dt, T):
    N  = int(round(1.0/dx))
    M = int(round(T/dt))
    x = np.arange(0, 1+dx, dx, dtype = float)
    t = np.arange(0, 1+dt, dt, dtype = float)
    matriz_u = np.ones((N, M), dtype = float)
    r  = alpha * dt / (dx**2)
    matriz_u[0, :] = funcion(x)
    for i in range(M):
        Un = U[n, :]
        U[n+1, 1:-1] = Un[1:-1] + r * (Un[2:] - 2.0*Un[1:-1] + Un[:-2])




    

    

def e_c_metodoimplicito():

In [2]:
import numpy as np
np.arange(0, 1+0.1, 0.1, dtype = float)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])